# Oanda Demo Trading Notebook

#### Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import winsound
import collections

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

#### API Setup

Read from config file

In [3]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

#### Retrive Data

In [4]:
# Get timestamp of the price and segregate it
def get_date_time(resp):
    
    time_stamp = resp['time']
    date_val, full_time = time_stamp.split(sep = 'T')
    time_val, time_fraction = full_time.split(sep = '.')
    
    return(date_val, time_val, time_fraction)

In [5]:
# Get bid and ask prices
def get_prices(resp):
    
    bid_price = float(resp['bids'][0]['price'])    
    ask_price = float(resp['asks'][0]['price'])
    spread = ask_price - bid_price
    
    return(bid_price, ask_price, spread)

In [6]:
# Terminate connection
def terminate_connection():
    try:
        print('Connection Terminated')
        r.terminate(message = "maxrecs records received")
    except:
        pass

In [75]:
def get_chunks(i, val):
    global chunk_size
    global small_list
    global size_flag
    global change_position
    
    if i < chunk_size: #Keep adding prices until we reach chunk size
        small_list.append(val)  
               
    if size_flag == 1:
        small_list.popleft()   #Remove the left most (first) price
        small_list.append(val) #Add current value to right most position
        
        change_position -= 1  #Price allocation in small list happens right to left so change position goes from chunk size to zero
        if change_position < 0: 
            change_position = chunk_size - 1 #If position reaches zero, it needs to be reset to chunk size - 1 (for index position)
    
    if len(small_list) == chunk_size: #check if small list has reached chunk size
        size_flag = 1  
    
    return(small_list)

In [76]:
def find_direction(lst):
    list_size = len(lst)
    val_dict = {}
    flag_val = 0

    for i in range(list_size): #write the list into a dictionary 
        val_dict[i] = lst[i]

    for i in range(list_size-1):
        if val_dict[i] < val_dict[i+1]:
            flag_val += 1

        elif val_dict[i] == val_dict[i+1]:
            flag_val += 0

        elif val_dict[i] > val_dict[i+1]:
            flag_val -= 1

    if flag_val == len(lst) - 1:
        direction = 'positive'
    elif flag_val == (len(lst) - 1)*-1:
        direction = 'negative'
    else:
        direction = 'nodir'
        
    return(flag_val, direction)

In [176]:
def find_direction1(lst):
    list_size = len(lst)
    val_dict = {}
    flag_val = 0
    diff = {}
    dir_flag = 'blank'
   
    for i in range(list_size-1): #write the list into a dictionary 
        diff[i] = lst[i+1] - lst[i]       
        
    if any(x==0 for x in diff.values()):
        dir_flag = 'nodir'
    elif all(x>0 for x in diff.values()):
        dir_flag = 'positive'
    elif all(x<0 for x in diff.values()):
        dir_flag = 'negative'
    else:
        dir_flag = 'dir_chg'

    return(dir_flag)

In [170]:
full_list = [4,0,0]

dir_flag, diff = find_direction1(full_list)
print(dir_flag)

nodir


In [177]:
full_list = [1,2,3,2,5,6,7,8,9,10,11,12,12,11,10,9,5,6,17,25,20,19,18]

chunk_size = 3
change_position = chunk_size - 1
small_list = collections.deque([])
size_flag = 0

for i, val in enumerate(full_list):
    
    small_list = get_chunks(i, val)
    d = find_direction1(small_list) 
    print(d)

positive
positive
positive
dir_chg
dir_chg
positive
positive
positive
positive
positive
positive
positive
nodir
nodir
negative
negative
negative
dir_chg
positive
positive
dir_chg
negative
negative


In [10]:
params = {
    'instruments': 'EUR_USD'
}
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

num = 4
bp_list = []
sp_list = []

for i, resp in enumerate(rv):
    if i < num:        
        print('Iteration : ',i)
        
        resp_type = resp['type']        
        
        if resp_type == 'PRICE':    
            date_val, time_val, time_fraction = get_date_time(resp)
            sell_price, buy_price, spread = get_prices(resp)          
            print('buy_price:',buy_price,',' , 'sell_price:',sell_price, ',', 'spread(pips):',spread*10000)
            print('----------------------------------------------')
            
        else:
            print('HEARTBEAT')

    else:
        terminate_connection()
        
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Iteration :  0
buy_price: 1.12925 , sell_price: 1.12906 , spread(pips): 1.900000000001345
----------------------------------------------
Iteration :  1
HEARTBEAT
Iteration :  2
HEARTBEAT
Iteration :  3
HEARTBEAT
Connection Terminated


# 2. fit into full code

In [39]:
params = {
          "instrument": "EUR_USD",
          "time": "2020-06-06T04:00:00Z"
        }
        
r = instruments.InstrumentsPositionBook(instrument="EUR_USD", params=params)
temp = api.request(r)
hist_price_list = temp["positionBook"]["buckets"]
print(len(hist_price_list))
temp

440


{'positionBook': {'instrument': 'EUR_USD',
  'time': '2020-06-06T04:00:00Z',
  'unixTime': '1591416000',
  'price': '1.12916',
  'bucketWidth': '0.00050',
  'buckets': [{'price': '1.03500',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0016'},
   {'price': '1.03600',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0016'},
   {'price': '1.03650',
    'longCountPercent': '0.0016',
    'shortCountPercent': '0.0016'},
   {'price': '1.03700',
    'longCountPercent': '0.0016',
    'shortCountPercent': '0.0032'},
   {'price': '1.03750',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0048'},
   {'price': '1.03800',
    'longCountPercent': '0.0048',
    'shortCountPercent': '0.0065'},
   {'price': '1.03850',
    'longCountPercent': '0.0065',
    'shortCountPercent': '0.0048'},
   {'price': '1.03900',
    'longCountPercent': '0.0081',
    'shortCountPercent': '0.0065'},
   {'price': '1.03950',
    'longCountPercent': '0.0097',
    'shortCountPercent':

In [46]:
params = {
          "instrument": "EUR_USD",
          "time": "2020-06-05T05:00:00Z"
        }
        
r = instruments.InstrumentsPositionBook(instrument="EUR_USD", params=params)
temp = api.request(r)
hist_price_list = temp["positionBook"]["buckets"]
print(len(hist_price_list))
temp

440


{'positionBook': {'instrument': 'EUR_USD',
  'time': '2020-06-05T05:00:00Z',
  'unixTime': '1591333200',
  'price': '1.13465',
  'bucketWidth': '0.00050',
  'buckets': [{'price': '1.03500',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0016'},
   {'price': '1.03600',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0016'},
   {'price': '1.03650',
    'longCountPercent': '0.0016',
    'shortCountPercent': '0.0016'},
   {'price': '1.03700',
    'longCountPercent': '0.0016',
    'shortCountPercent': '0.0032'},
   {'price': '1.03750',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0048'},
   {'price': '1.03800',
    'longCountPercent': '0.0048',
    'shortCountPercent': '0.0064'},
   {'price': '1.03850',
    'longCountPercent': '0.0064',
    'shortCountPercent': '0.0048'},
   {'price': '1.03900',
    'longCountPercent': '0.0080',
    'shortCountPercent': '0.0064'},
   {'price': '1.03950',
    'longCountPercent': '0.0096',
    'shortCountPercent':

In [54]:
params = {
          "from": "2020-05-01T00:00:00Z",
          "to": "2020-05-01T01:00:00Z",
          "granularity": "S5"}

r = instruments.InstrumentsCandles(instrument="EUR_USD",
                                   params=params)

api.request(r)

{'instrument': 'EUR_USD',
 'granularity': 'S5',
 'candles': [{'complete': True,
   'volume': 7,
   'time': '2020-05-01T00:00:00.000000000Z',
   'mid': {'o': '1.09448', 'h': '1.09450', 'l': '1.09445', 'c': '1.09445'}},
  {'complete': True,
   'volume': 2,
   'time': '2020-05-01T00:00:05.000000000Z',
   'mid': {'o': '1.09444', 'h': '1.09444', 'l': '1.09442', 'c': '1.09442'}},
  {'complete': True,
   'volume': 2,
   'time': '2020-05-01T00:00:10.000000000Z',
   'mid': {'o': '1.09440', 'h': '1.09442', 'l': '1.09440', 'c': '1.09442'}},
  {'complete': True,
   'volume': 1,
   'time': '2020-05-01T00:00:15.000000000Z',
   'mid': {'o': '1.09440', 'h': '1.09440', 'l': '1.09440', 'c': '1.09440'}},
  {'complete': True,
   'volume': 3,
   'time': '2020-05-01T00:00:25.000000000Z',
   'mid': {'o': '1.09438', 'h': '1.09438', 'l': '1.09436', 'c': '1.09436'}},
  {'complete': True,
   'volume': 7,
   'time': '2020-05-01T00:00:40.000000000Z',
   'mid': {'o': '1.09434', 'h': '1.09439', 'l': '1.09432', 'c': 

In [61]:
df = pd.read_csv('DAT_NT_EURUSD_T_BID_202005.csv', sep = ';', header = None, names = ['timestamp', 'price', 'unknown'])

In [62]:
df.head()

,timestamp,price,unknown
0,20200501 000000,1.09450,0
1,20200501 000002,1.09448,0
2,20200501 000003,1.09447,0
3,20200501 000006,1.09445,0
4,20200501 000027,1.09446,0
